In [1]:
import sys
sys.path.append(r'../')

In [2]:
from dolfin import *

import cell_geom as ce
import cell_material as ma
import cell_computation as com

In [3]:
# Set geometry
mesh = Mesh(r"../m.xml")
cell = ce.UnitCell(mesh)
inc = ce.InclusionCircle(2, (0.5, 0.5), 0.25)
inc_di = {'circle_inc': inc}
cell.set_append_inclusion(inc_di)

In [4]:
VFS = VectorFunctionSpace(cell.mesh, "CG", 1,
                        constrained_domain=ce.PeriodicBoundary_no_corner(2))
FS = FunctionSpace(cell.mesh, "CG", 1,
                       constrained_domain=ce.PeriodicBoundary_no_corner(2))

In [5]:
# Set materials
E_m, nu_m, Kappa_m = 2e5, 0.4, 7.
n = 1000
E_i, nu_i, Kappa_i = 1000*E_m, 0.3, n*Kappa_m

mat_m = ma.neo_hook_eap(E_m, nu_m, Kappa_m)
mat_i = ma.neo_hook_eap(E_i, nu_i, Kappa_i)
mat_li = [mat_m, mat_i]

In [6]:
# Macro Field Boundary
F_bar = [0.9, 0.,
             0., 1.]
E_bar = [0., 0.2]

In [7]:
w = Function(VFS)
el_pot_phi = Function(FS)
strain_space_w = TensorFunctionSpace(cell.mesh, 'DG', 0)
strain_space_E = VectorFunctionSpace(cell.mesh, 'DG', 0)

def deform_grad_with_macro(F_bar, w_component):
    return F_bar + grad(w_component)

def e_field_with_macro(E_bar, phi):
    # return E_bar + grad(phi)
    return E_bar - grad(phi)

In [8]:
# Computation Initialization
comp = com.MicroComputation(cell, mat_li,
                            [deform_grad_with_macro, e_field_with_macro],
                            [strain_space_w, strain_space_E])

comp.input([F_bar, E_bar], [w, el_pot_phi])

In [9]:
com.set_solver_parameters('non_lin_newton', 'iterative')

.-------------------.
| Solver Parameters |
.-------------------.
iterative method is used
a valid preconditioner should be provided


In [10]:
comp.comp_fluctuation()

fluctuation computation finished


In [11]:
# Post-Processing
# comp._energy_update()
# comp.comp_strain()
# comp.comp_stress()
# comp.avg_merge_strain()
# comp.avg_merge_stress()
com.set_post_solver_parameters(lin_method='iterative',)
comp.effective_moduli_2()

+----------------------------+
| Post Processing Parameters |
+----------------------------+
iterative method is used
a valid preconditioner should be provided
strain computation finished
average merge moduli computation finished


array([[  8.47401634e+04,   2.60643162e+01,   3.12208885e+01,
          4.94745351e+04,   3.26332515e-15,   1.44547980e-11],
       [  2.60643162e+01,   5.38339945e+04,   6.48049691e+04,
         -2.00799651e+00,   1.41317502e-11,   3.28935774e-15],
       [  3.12208885e+01,   6.48049691e+04,   6.30214375e+04,
         -4.26559107e+00,   1.57019447e-11,   3.65484191e-15],
       [  4.94745351e+04,  -2.00799651e+00,  -4.26559107e+00,
          9.55184214e+04,   4.39895674e-15,   1.81228396e-11],
       [  3.26332515e-15,   1.41317502e-11,   1.57019447e-11,
          4.39895674e-15,  -4.44858493e-11,  -7.39574530e-16],
       [  1.44547980e-11,   3.28935774e-15,   3.65484191e-15,
          1.81228396e-11,  -7.39574531e-16,  -4.39689106e-11]])